In [1]:
import numpy as np
import random
import json
import nltk
import string
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, Dropout,LSTM
import matplotlib.pyplot as plt

In [2]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()

In [6]:
data_file = open('Intents.json','r')
data = json.load(data_file)
print(data)

{'Intents': [{'tag': 'greeting', 'patterns': ['hello', 'hi', 'hello chatbot', 'hi chatbot'], 'responses': ['hello , I am here to answer all your question abou machine learning']}, {'tag': 'name', 'patterns': ["what's your name ?", 'can you tell me your name ?'], 'responses': ['I am Chatty', 'My name is Chatty']}, {'tag': 'question', 'patterns': ['I have a question', 'can you answer my question', 'can I ask you'], 'responses': ['yes Of course, my mission is to answer all your questions about Machine learning']}, {'tag': 'Machine learning', 'patterns': ['What is machine learning', 'can you tell me about machine learning'], 'responses': ['the use and development of computer systems that are able to learn and adapt without following explicit instructions, by using algorithms and statistical models to analyse and draw inferences from patterns in data.']}, {'tag': 'MLtypes', 'patterns': ['What are types of Machine learning ?'], 'responses': ['These are three types of machine learning: superv

In [7]:
words=[]  #for storing words existing in patterns
classes=[] #for storing tags
data_x=[] #for storing patterns
data_y=[] #for storing tags for each pattern in data_x

In [8]:
#fill lists above from json file
for intent in data["Intents"]:
    for pattern in intent["patterns"]:
        tokens=nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"])
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [9]:
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation] #lemmetize words and convert them to lower case

In [10]:
print(words)

['hello', 'hi', 'hello', 'chatbot', 'hi', 'chatbot', 'what', "'s", 'your', 'name', 'can', 'you', 'tell', 'me', 'your', 'name', 'i', 'have', 'a', 'question', 'can', 'you', 'answer', 'my', 'question', 'can', 'i', 'ask', 'you', 'what', 'is', 'machine', 'learning', 'can', 'you', 'tell', 'me', 'about', 'machine', 'learning', 'what', 'are', 'type', 'of', 'machine', 'learning', 'what', 'is', 'supervised', 'learning', 'what', 'are', 'algorithm', 'of', 'supervised', 'learning', 'what', 'is', 'unsupervised', 'learning', 'what', 'are', 'algorithm', 'of', 'unsupervised', 'learning', 'what', 'is', 'reinforcement', 'learning', 'what', 'are', 'algorithm', 'of', 'reinforcement', 'learning', 'ok', 'goodbye', 'thank', 'you', 'can', 'you', 'tell', 'me', 'where', 'to', 'learn', 'machine', 'learning', 'programming', 'where', 'to', 'learn', 'machine', 'learning', 'how', 'old', 'are', 'you', 'what', "'s", 'your', 'age', 'i', 'will', 'try', 'yes', 'of', 'course']


In [11]:
#sort words and classes in alphabetical orders
words=sorted(set(words))
classes=sorted(set(classes))

In [12]:
print(data_x)
print(len(data_x))

['hello', 'hi', 'hello chatbot', 'hi chatbot', "what's your name ?", 'can you tell me your name ?', 'I have a question', 'can you answer my question', 'can I ask you', 'What is machine learning', 'can you tell me about machine learning', 'What are types of Machine learning ?', 'What is supervised learning ?', 'What are algorithms of supervised learning ?', 'What is unsupervised learning ?', 'What are algorithms of unsupervised learning ?', 'What is reinforcement learning ?', 'What are algorithms of reinforcement learning ?', 'OK,goodbye', 'Thank you', 'Can you tell me where to learn machine learning programming', 'where to learn machine learning', 'How old are you?', "what's your age", 'I will try', 'yes of course']
26


In [13]:
#implementing bag of words model bow
training=[]
out_empty=[0]*len(classes)
for index, information in enumerate(data_x):
    bow=[]
    text=lemmatizer.lemmatize(information.lower())
    print(text)
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row=list(out_empty)
    output_row[classes.index(data_y[index])]=1
    training.append([bow,output_row])
random.shuffle(training)
training=np.array(training,dtype=object)
train_x=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))
print(words)
print(len(words))
print(train_x)
print(len(train_x[0]))

hello
hi
hello chatbot
hi chatbot
what's your name ?
can you tell me your name ?
i have a question
can you answer my question
can i ask you
what is machine learning
can you tell me about machine learning
what are types of machine learning ?
what is supervised learning ?
what are algorithms of supervised learning ?
what is unsupervised learning ?
what are algorithms of unsupervised learning ?
what is reinforcement learning ?
what are algorithms of reinforcement learning ?
ok,goodbye
thank you
can you tell me where to learn machine learning programming
where to learn machine learning
how old are you?
what's your age
i will try
yes of course
["'s", 'a', 'about', 'age', 'algorithm', 'answer', 'are', 'ask', 'can', 'chatbot', 'course', 'goodbye', 'have', 'hello', 'hi', 'how', 'i', 'is', 'learn', 'learning', 'machine', 'me', 'my', 'name', 'of', 'ok', 'old', 'programming', 'question', 'reinforcement', 'supervised', 'tell', 'thank', 'to', 'try', 'type', 'unsupervised', 'what', 'where', 'will', 

In [14]:
#building the neural network model
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(68,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))
adam=tf.keras.optimizers.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5632      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 68)                8772      
_________________________________________________________________
dropout_1 (Dropout)          (None, 68)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                966       
Total params: 15,370
Trainable params: 15,370
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
#training our model
history=model.fit(x=train_x,y=train_y,batch_size=10,epochs=200,verbose=1)
print(history.history.keys())

Epoch 1/200
3/3 [==============================] - 2s 16ms/step - loss: 2.6843 - accuracy: 0.0385
Epoch 2/200
3/3 [==============================] - 0s 4ms/step - loss: 2.7187 - accuracy: 0.0385
Epoch 3/200
3/3 [==============================] - 0s 5ms/step - loss: 2.6935 - accuracy: 0.0385
Epoch 4/200
3/3 [==============================] - 0s 5ms/step - loss: 2.6937 - accuracy: 0.1538
Epoch 5/200
3/3 [==============================] - 0s 5ms/step - loss: 2.5994 - accuracy: 0.0385
Epoch 6/200
3/3 [==============================] - 0s 5ms/step - loss: 2.5085 - accuracy: 0.1154
Epoch 7/200
3/3 [==============================] - 0s 6ms/step - loss: 2.4896 - accuracy: 0.1154
Epoch 8/200
3/3 [==============================] - 0s 5ms/step - loss: 2.5157 - accuracy: 0.2308
Epoch 9/200
3/3 [==============================] - 0s 5ms/step - loss: 2.4803 - accuracy: 0.1923
Epoch 10/200
3/3 [==============================] - 0s 4ms/step - loss: 2.4505 - accuracy: 0.2308
Epoch 11/200
3/3 [==========

3/3 [==============================] - 0s 4ms/step - loss: 0.6229 - accuracy: 0.8846
Epoch 84/200
3/3 [==============================] - 0s 4ms/step - loss: 0.6379 - accuracy: 0.8846
Epoch 85/200
3/3 [==============================] - 0s 4ms/step - loss: 0.6660 - accuracy: 0.8077
Epoch 86/200
3/3 [==============================] - 0s 7ms/step - loss: 0.6862 - accuracy: 0.8077
Epoch 87/200
3/3 [==============================] - 0s 4ms/step - loss: 0.5528 - accuracy: 0.8462
Epoch 88/200
3/3 [==============================] - 0s 4ms/step - loss: 0.6211 - accuracy: 0.9231
Epoch 89/200
3/3 [==============================] - 0s 4ms/step - loss: 0.5630 - accuracy: 0.8462
Epoch 90/200
3/3 [==============================] - 0s 5ms/step - loss: 0.5904 - accuracy: 0.8846
Epoch 91/200
3/3 [==============================] - 0s 4ms/step - loss: 0.7037 - accuracy: 0.8077
Epoch 92/200
3/3 [==============================] - 0s 5ms/step - loss: 0.6340 - accuracy: 0.8077
Epoch 93/200
3/3 [===============

3/3 [==============================] - 0s 5ms/step - loss: 0.2876 - accuracy: 1.0000
Epoch 166/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1457 - accuracy: 0.9615
Epoch 167/200
3/3 [==============================] - 0s 6ms/step - loss: 0.2141 - accuracy: 0.9615
Epoch 168/200
3/3 [==============================] - 0s 5ms/step - loss: 0.2437 - accuracy: 0.9231
Epoch 169/200
3/3 [==============================] - 0s 6ms/step - loss: 0.2020 - accuracy: 0.9615
Epoch 170/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1526 - accuracy: 0.9615
Epoch 171/200
3/3 [==============================] - 0s 4ms/step - loss: 0.1931 - accuracy: 0.9615
Epoch 172/200
3/3 [==============================] - 0s 4ms/step - loss: 0.1545 - accuracy: 0.9615
Epoch 173/200
3/3 [==============================] - 0s 5ms/step - loss: 0.1819 - accuracy: 0.9615
Epoch 174/200
3/3 [==============================] - 0s 4ms/step - loss: 0.2087 - accuracy: 0.9615
Epoch 175/200
3/3 [=====

In [16]:
#generating responses for unknown questions
def random_response():
    random_list=["Please try to write something more descriptive",
                "I am terribly sorry , I can't quit catch that",
                "I can't answer this yet, please try something else"]
    list_count=len(random_list)
    random_item=random.randrange(list_count)
    return random_list[random_item]

In [17]:
#tokenizing and lemmatizing a text 
def Clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word.lower()) for word in tokens if word not in string.punctuation]
    return tokens

In [18]:
#converting a text to a bag of words
def Bag_of_words(text, vocab) :
    tokens=Clean_text(text)
    bow=[0]*len(vocab)
    for word in tokens:
        for index,information in enumerate(vocab):
            if word==information :
                bow[index]=1
    return np.array(bow)

In [19]:
#predict the class of user input
def Pred_class(text, vocab, labels):
    bow=Bag_of_words(text,vocab)
    result=model.predict(np.array([bow]))[0]
    thresh=0.5
    y_pred=[[index,res] for index,res in enumerate(result) if res>thresh]
    y_pred.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

In [20]:
#return a response 
def Get_response(intents_list, intents_json):
    result=""
    if len(intents_list)==0:
        random_response()
    else:
        tag=intents_list[0]
        list_of_intents=intents_json["Intents"]
        for i in list_of_intents:
            if i["tag"]==tag:
                result=random.choice(i["responses"])
                break
    return result
        

In [ ]:
while True :
    message=input("")
    if message=="quit":
        break
    intents=Pred_class(message,words,classes)
    if len(intents)>0:
        result=Get_response(intents,data)
    else:
       result=random_response()
    print(result)
        

hello
hello , I am here to answer all your question abou machine learning
what's your name ?
My name is Chatty
i have a question
yes Of course, my mission is to answer all your questions about Machine learning
what is machine learning
the use and development of computer systems that are able to learn and adapt without following explicit instructions, by using algorithms and statistical models to analyse and draw inferences from patterns in data.
what is supervised learning
I can't answer this yet, please try something else
what's supervised learning
Please try to write something more descriptive
what is your age
believe me I don't know, can you help me to know my age?
yes of course
Thank you dude !
where can i learn machine learning
You can learn on coursera and udemy
what's unsupervised learning
I can't answer this yet, please try something else
What is reinforcement learning ?
Reinforcement Learning (RL) is the science of decision making. It is about learning the optimal behavior in 